Extraction

In [ ]:
import os
import pandas as pd
csv_path = "../../data/spanish/spanish_tweets.csv"
df = pd.read_csv(csv_path)
df.head()

AttributeError: module 'torch' has no attribute '__version__'

Cleaning data

In [8]:
import pandas as pd
import numpy as np
# removing usernames
df["clean_text"] = df["text"].str.replace(r"@\w+", "", regex=True).str.strip()
# removing urls
df["clean_text"] = df["clean_text"].str.replace(r"http\S+", "", regex=True)
# removing empty texts
df = df[df["clean_text"].str.len() > 0]
df["clean_text"].head()

df["sentiment"].unique()
#mapping sentiment labels to numbers
mapping = {'sad': 0, 'mad': 1, 'scared': 2, 'joyful': 3, 'peaceful': 4, 'powerful': 5}
df["sentiment_labels"] = df['sentiment'].map(mapping)
df.tail()


,user,text,date,emotion,sentiment,clean_text,sentiment_labels
2585,@lavivianaleyva,No podemos vivir con miedo: ¡Manejen borrach...,"Jan 6, 2024 · 3:08 AM UTC",daring,joyful,No podemos vivir con miedo: ¡Manejen borrach...,3
2586,@Carmeen_Alicia,"La vida es un constante, SIN MIEDO AL ÉXITO 💅🏼","Jan 6, 2024 · 3:07 AM UTC",daring,joyful,"La vida es un constante, SIN MIEDO AL ÉXITO 💅🏼",3
2587,@homicidios_,Esquizofrenia = mente dividida: Miedo a las re...,"Jan 6, 2024 · 2:59 AM UTC",daring,joyful,Esquizofrenia = mente dividida: Miedo a las re...,3
2588,@brigethcoba,"""Lo que más miedo me da, es ver cómo desaparec...","Jan 6, 2024 · 2:55 AM UTC",daring,joyful,"""Lo que más miedo me da, es ver cómo desaparec...",3
2589,@nanyfrias11,Saltando de apoco ala pile sin agua xd sin mie...,"Jan 6, 2024 · 2:54 AM UTC",daring,joyful,Saltando de apoco ala pile sin agua xd sin mie...,3


In [12]:
from transformers import pipeline, AutoTokenizer, BertTokenizer
import torch
from sklearn.model_selection import train_test_split
#Splitting data into train and validation sets
train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df["sentiment_labels"],
    random_state=42
)
#Model setup
model_name = "bert-base-multilingual-uncased"
tok = BertTokenizer.from_pretrained(model_name)

MAX_LEN = 100  # tweets are short
#Converting to tensors
def tokenize(batch):
    return tok(
        batch["clean_text"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_encodings = tokenize(train_df)
val_encodings = tokenize(val_df)
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')
unmasker("Hello I'm a [MASK] model.") 




ImportError: DLL load failed while importing _C: The specified module could not be found.

Conversion to Dataset

In [10]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SentimentDataset(train_encodings, train_df["sentiment_labels"].values)
val_dataset = SentimentDataset(val_encodings, val_df["sentiment_labels"].values)

NameError: name 'torch' is not defined